# install & import libraries

In [ ]:
!pip install lightgbm==3.3.5

In [ ]:
!pip install sktime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 10.9 MB/s eta 0:00:00


In [ ]:
#필요한 library import
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm as tq
from tqdm import tqdm
import sktime

from scipy.stats import spearmanr

print("tqdm version: {}".format(tq.__version__))

import warnings
warnings.filterwarnings('ignore')

tqdm version: 4.66.1


In [ ]:
import sklearn
#from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

#from time import gmtime, strftime
#import gc

#from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import ExtraTreesRegressor
#from lightgbm import LGBMRegressor
#from sklearn.metrics import (roc_curve, auc, accuracy_score, mean_absolute_error)

from sktime.forecasting.model_selection import temporal_train_test_split
#from sktime.utils.plotting import plot_series

In [ ]:
import os
SEED = 2023
np.random.seed(SEED)

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


# data load

In [ ]:
train = pd.read_csv('/gdrive/MyDrive/dacon/data/train_final.csv' )
test = pd.read_csv('/gdrive/MyDrive/dacon/data/test_final.csv' )

In [ ]:
train.head()

,num,date_time,power,prec,wind,hum,temp,day,week,day_hour_mean,day_hour_std,holiday,sin_time,cos_time,THI,CDH,solar_flag,month,senstemp
0,1,2022-06-01 00:00:00,1085.28,0.0,0.9,42.0,18.6,2,22,1774.744615,517.982222,1,0.000000,1.000000,49.6576,-5.4,0,6,17.87
1,1,2022-06-01 01:00:00,1047.36,0.0,1.1,45.0,18.0,2,22,1687.347692,500.769931,1,0.258819,0.965926,47.7625,-11.4,0,6,17.52
2,1,2022-06-01 02:00:00,974.88,0.0,1.5,45.0,17.7,2,22,1571.483077,465.227458,1,0.500000,0.866025,47.2225,-17.7,0,6,17.24
3,1,2022-06-01 03:00:00,953.76,0.0,1.4,48.0,16.7,2,22,1522.153846,436.601091,1,0.707107,0.707107,44.7856,-25.0,0,6,16.49
4,1,2022-06-01 04:00:00,986.40,0.0,2.8,43.0,18.4,2,22,1506.793846,405.518091,1,0.866025,0.500000,49.0061,-30.6,0,6,17.76


In [ ]:
test.head()

,num,date_time,prec,wind,hum,temp,day,week,day_hour_mean,day_hour_std,holiday,sin_time,cos_time,THI,CDH,solar_flag,month,senstemp
0,1,2022-08-25 00:00:00,0.0,2.2,72,23.5,3,34,1627.80,446.984913,0,0.000000,1.000000,58.3456,-0.5,0,8,24.86
1,1,2022-08-25 01:00:00,0.0,0.9,72,23.0,3,34,1550.08,449.091398,0,0.258819,0.965926,57.4456,-1.5,0,8,24.36
2,1,2022-08-25 02:00:00,0.0,1.5,75,22.7,3,34,1431.12,415.453568,0,0.500000,0.866025,57.8725,-2.8,0,8,24.27
3,1,2022-08-25 03:00:00,0.0,1.3,78,22.1,3,34,1372.20,378.117772,0,0.707107,0.707107,57.9376,-4.7,0,8,23.87
4,1,2022-08-25 04:00:00,0.0,1.0,77,21.8,3,34,1381.72,360.348483,0,0.866025,0.500000,56.9961,-6.9,0,8,23.50


# hyperparams tuning

## all

In [ ]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.metrics import make_scorer

# Define SMAPE loss function
def SMAPE(true, pred):
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred))) * 100

smape = make_scorer(SMAPE, greater_is_better = False)

In [ ]:
features = train.columns.values[3:]

## grid search

In [ ]:
parameters = {
    'n_estimators' : [100, 200],
    'min_samples_leaf': [2, 5,10,20, 34],
    'max_features': [2,3,4, 8, 13]
}

df = pd.DataFrame(columns = ['n_estimators', 'min_samples_leaf','max_features'])
for i in tqdm(np.arange(39, 40)):
    y = train.loc[train.num == i, 'power']
    x = train.loc[train.num == i, features]
    # 마지막 일주일 발전량을 validset으로 24시간*7일 = 168
    y_train, y_test, x_train, x_test = temporal_train_test_split(y = y, X = x, test_size = 168)


    pds = PredefinedSplit(np.append(-np.ones(len(x)-168), np.zeros(168)))
    gcv = GridSearchCV(estimator = ExtraTreesRegressor(verbose = 0, criterion = 'absolute_error', random_state = 42),
                       param_grid = parameters, scoring = smape, cv = pds, refit = True, verbose = False)
    gcv.fit(x, y)
    params = gcv.best_params_
    best = gcv.best_estimator_
    print(params)
    pred = best.predict(x_test)
    building = 'building'+str(i)
    print(building + '|| SMAPE : {}'.format(SMAPE(y_test, pred)))
    df = pd.concat([df, pd.DataFrame(params, index = [0])], axis = 0)
    df.to_csv('/gdrive/MyDrive/dacon/xt/mid_xtr_params.csv')

100%|██████████| 1/1 [05:10<00:00, 310.40s/it]

{'max_features': 13, 'min_samples_leaf': 20, 'n_estimators': 100}
building39|| SMAPE : 1.4536373992745857


In [ ]:
et_params = pd.read_csv('/gdrive/MyDrive/dacon/xt/xtr_params_add.csv', index_col = 0)
et_params.head()

,n_estimators,min_samples_leaf,max_features
0,200,5,13
1,200,2,13
2,100,34,8
3,200,34,8
4,100,10,13


# test data predict

In [ ]:
preds = np.array([])
for i in tqdm(range(100)):

    y_train = train.loc[train.num == i+1, 'power']
    x_train, x_test = train.loc[train.num == i+1, features], test.loc[test.num == i+1, features]
    x_test = x_test[x_train.columns]

    et = ExtraTreesRegressor(n_estimators = et_params.iloc[i, 0], min_samples_leaf = et_params.iloc[i, 1], max_features = et_params.iloc[i, 2],
                                criterion = 'absolute_error', random_state = 42, verbose = 0)

    # if xgb_params.iloc[i,6] != 0:  # 만약 alpha가 0이 아니면 weighted_mse 사용
    #     xgb.set_params(**{'objective':weighted_mse(xgb_params.iloc[i,6])})

    et.fit(x_train, y_train)
    y_pred = et.predict(x_test)
    preds = np.append(preds, y_pred)

100%|██████████| 100/100 [16:57<00:00, 10.17s/it]


In [ ]:
submission = pd.read_csv('/gdrive/MyDrive/dacon/data/sample_submission.csv')
submission['answer'] = preds
submission.to_csv('/gdrive/MyDrive/dacon/submission/sub_0826et.csv', index = False)